# Custom I2C master demo using MicroBlaze and evaluating bytes transferred

**One problem with the standard PYNQ library is that it does not return the length of bytes transferred to/from the slave. Modifying the library, as shown later, will overcome this missing feature. This demo presents three different ways of how to use the PMODA port to communicate with a INA219 current sensor from Adafruit via I2C.**

(The demos were carried out on a PYNQ-Z1 with v2.1.)

***

## Modifications for method 1 and 2

To run the first two methods and being able to determine how many bytes where transfered, the file ```i2c.h``` has to be modified. This can be done directly on the SD card (by starting a terminal in jupyter) and modify the file on path ```<PYNQ repository>/pynq/lib/pmod/bsp_iop_pmod/iop_pmod_mb/include/i2c.h```. Change the return value type of the read/write functions from ```void``` to ```unsigned int``` is all to return the bytes transfered via I2C:

```C
unsigned int i2c_read(i2c dev_id, unsigned int slave_address,
              unsigned char* buffer, unsigned int length);
unsigned int i2c_write(i2c dev_id, unsigned int slave_address,
               unsigned char* buffer, unsigned int length);
```

### Method 1: I2C communication from Python directly (slowest)

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary

base = BaseOverlay('base.bit')

lib = MicroblazeLibrary(base.PMODA, ['i2c', 'pmod_grove'])

device = lib.i2c_open(lib.PMOD_G4_B, lib.PMOD_G4_A)

In [2]:
# Set Calibration register value and read back its value

buf = bytearray(3)
buf[0] = 0
buf[1] = 80
buf[2] = 0
len_bytes = 0xAA
len_bytes = device.write(0x40, buf, 3)
print(len_bytes)
len_bytes = device.read(0x40, buf, 2)
print(len_bytes)

((buf[0] & 0xFF) << 8) | buf[1] # value returned from INA219

3
2


4096

### Method 2: I2C communication from Python using MicroBlaze magic (faster)

In [3]:
%%microblaze base.PMODA
#include <i2c.h>
#include <pmod_grove.h>
#include <pyprintf.h>

i2c device;

int ina219_open() {
    device = i2c_open(PMOD_G4_B, PMOD_G4_A);
}

int ina219_read(unsigned char addr) {
    unsigned char buf[2];
    buf[0] = addr;
    i2c_write(device, 0x40, buf, 1); // read pointer set to register
    i2c_read(device, 0x40, buf, 2);
    return ((buf[0]) << 8) | buf[1];
}

int ina219_write(unsigned char addr, unsigned char msbyte, unsigned char lsbyte) {
    unsigned char buf[3];
    unsigned int len = 0xAA; // dummy value to test for length transfered
    
    buf[0] = addr;
    buf[1] = msbyte;
    buf[2] = lsbyte;
    len = i2c_write(device, 0x40, buf, 3); // write pointer set to register
    pyprintf("Bytes written: %d\n", len);
    
    len = 0xAA; // Reset to known value
    len = i2c_read(device, 0x40, buf, 2);
    pyprintf("Bytes read: %d\n", len);

    return ((buf[0]) << 8) | buf[1];    
}

int print_hex(unsigned int val) {
    pyprintf("Values: 0x%x\n", val);
    return 0;
}

In [4]:
ina219_open()

0

In [5]:
ret = ina219_write(0, 80, 0)
print_hex(ret)

Bytes written: 3
Bytes read: 2
Values: 0x1000


0

In [6]:
config = ina219_read(0)
print_hex(config)

Values: 0x1000


0

## Modifications to use method 3

Using method requires deeper insight in how to write a custom application for the MicroBlaze system:
https://pynq.readthedocs.io/en/v2.2.1/overlay_design_methodology/pynq_microblaze_subsystem.html

This requires a couple of steps to be performed before using the example (unless you just download the pre-compiled binary and the Python code to the Pynq board).

*Please do NOT confuse this procedure with "Creating a New PYNQ Microblaze" which creates a new softprocessor. This means it will create a new IOP instance with PYNQ MicroBlaze Subsystems. Instead, here we use one of the three IOPs (PMODA, PMODB, Arduino) already available.*


**To be able to create a custom pre-compiled MicroBlaze binary file please refer to the [prerequisites](#Prerequisites-to-use-method-3) before proceeding.**


Our custom project, e.g. ```<PYNQ repository>/pynq/lib/pmod/pmod_custom_i2c/``` contains two folders:
```
Debug/ -> makefile, linker scripts, etc.
src/   -> actual source code
```
(This custom example project including the MicroBlaze and Python code can be found in the ```pmod_custom folder```.)

Workflow for creating a pre-compiled MicroBlaze binary:

1. Source SDK toolchain settings to put the MicroBlaze gcc compiler to the path: ```$ source /opt/Xilinx/SDK/2017.4/settings64.sh```
2. Make sure the custom project is adapted (makefiles, etc. as described in the link above)
3. Modify the functions on the host machine ```<PYNQ repository>/pynq/lib/pmod/bsp_iop_pmod/iop_pmod_mb/include/i2c.h``` to return a value of the type ```unsigned int```
4. Compile the code with ```make``` in the ```Debug/``` folder.


After compilation the binary can be downloaded to the PYNQ-Z1 board via SCP and the default password `xilinx`
```
$ scp pmod_custom_i2c.bin xilinx@192.168.2.99:/home/xilinx/pynq/lib/pmod
```

Now we have the pre-compiled PYNQ MicroBlaze driver on the Pynq board, the Python class must be located on the Pynq board ```/home/xilinx/pynq/lib/pmod/pmod_custom_i2c.py``` to communicate with it.

### Method 3: Custom (pre-compiled) MicroBlaze binary (faster)

In [7]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

In [8]:
from pynq.lib.pmod import Custom_I2C
from pynq.lib.pmod import PMOD_GROVE_G4

c = Custom_I2C(base.PMODA, PMOD_GROVE_G4)

In [9]:
print(c.get_value())

2


## Prerequisites to use method 3

In order to compile code for the MicroBlaze processor please follow the process described here.

Please also refer to the ```<PYNQ repository/sdbuild/README.md``` for futher information for the host setup.

### Install Ubuntu and VirtualBox

* Install Oracle VM VirtualBox
* Install Ubuntu 16.04 LTS with automatic login


```
$ sudo adduser <username> sudo
$ sudo adduser <username> vboxsf
```

* Install 32-bit libraries required for later Vivado installation:

```
$ sudo dpkg --add-architecture i386
$ sudo apt-get udpate
$ sudo apt-get install libz1:386 libncurses5:i386 libbz2-1.0:i386 libstdc++6:i386 
```

### Install Xilinx SDK

* Download Xilinx Vivado SDK WebPack 2017.4 (Xilinx_Vivado_SDK_Web_2017.4_1216_1_Lin64.bin)
* For ease of usage create a folder on /home directory with a symbolic link to the default directory where Xilinx want to be installed.

```
sudo mkdir /home/Xilinx
sudo chmod 777 /home/Xilinx
sudo ln -s /home/Xilinx /opt/Xilinx
```

* Install the SDK as described by the Xilinx User Guide to ```/opt/Xilinx```

### Setup host for PYNQ

* Modify script ```<PYNQ repository>/sdbuild/scripts/setup_host.sh``` before executing: Substitute ```lib32bz2-1.0``` with ```libbz2-1.0:i386```
* The ```<PYNQ repositoy>``` must be on a local path, not on the shared folder. Otherwise it will fail!
* Please note: Executing the script assumes a host with a passwordless sudo.